LDA = latennt dirchalrect allocation - collects topics from 
WORD2Vec - similar to PCA,builds latent features in text
Ngraming - successions of roots
NLTK parts of speech tagging clasify just adjactives and nouns

In [1]:
!rm -r metastore_db

In [2]:
import re

import pyspark as ps    # for the pyspark suite
import os               # for environ variables in Part 3

%load_ext autoreload
%autoreload 2

spark = ps.sql.SparkSession.builder \
            .appName("yelp_explore") \
            .getOrCreate()
        
import numpy as np
import pandas as pd

from nltk.stem.porter import PorterStemmer

In [3]:
## none of this works
# from src import build_df

# from pyspark import SparkContext, SparkConf
# from pyspark.sql import functions as F

# sc = SparkContext

# x.saveAsTextFile("s3n://b-datasets/flight_data2/")

# url = 'https://s3-us-west-2.amazonaws.com/wanderwell-ready/yelp_academic_dataset_review.json'

# ps.SparkContext.textFile?

# sc = spark.sparkContext

# # Load a text file and convert each line to a Row.
# lines = sc.textFile(url)
# parts = lines.map(lambda l: l.split(","))
# # Each line is converted to a tuple.
# people = parts.map(lambda p: (p[0], p[1].strip()))

# # Initialize the session
# spark = SparkSession.builder.appName("using_s3").getOrCreate()



# # Create an RDD from a file in S3 using the s3n protocol
# s3nRdd = spark.sparkContext.textFile(url)

# shityelp = spark.createDataFrame(s3nRdd)

In [6]:
ps.SparkContext.textFile('s3n://wanderwell-ready/yelp_academic_review.json')

TypeError: unbound method textFile() must be called with SparkContext instance as first argument (got str instance instead)

In [7]:
spark.read.json('s3n://wanderwell-ready/yelp_academic_review.json')

AnalysisException: u'Path does not exist: s3n://wanderwell-ready/yelp_academic_review.json;'

In [46]:
yelp = spark.read.json('data/yelp_academic_reviews.json')

In [47]:
yelp.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [54]:
yelp.registerTempTable('yelp')

corpus =spark.sql('''
        SELECT business_id, text, stars, cool + funny + useful as popular , user_id 
        FROM yelp
        limit 10000
       ''' )


In [61]:
corpus.registerTempTable('corpus')

high = spark.sql('''
        SELECT business_id, text, stars, popular , user_id 
        FROM corpus
        WHERE stars > 3
''')

low = spark.sql('''
        SELECT business_id, text, stars, popular , user_id 
        FROM corpus
        WHERE stars < 4
        ''')

#### it looks like somone has given some credibility to 33% of the bad reviews.

In [87]:
good_n_low =  low.filter('popular > 1')
float(good_n_low.count()) / low.count()  

0.33312040881507504

In [72]:
float(high.count())/low.count()  #  there are 2.2 times as many positive reviews as negative reviews.

2.1938677738741617

##### distribution between credible neg reviews and credible pos reviws is similar

In [126]:
good_n_high =  high.filter('popular > 1')
float(good_n_high.count()) / high.count()  

0.2793710874945407

In [150]:
users = corpus.groupBy(col: 'user_id')

SyntaxError: invalid syntax (<ipython-input-150-86ebea0334f1>, line 1)

In [148]:
uagg = users.agg({'stars' : 'mean'})

In [80]:
spark.udf.register('labels', lambda x: 0 if x < 4 else 1)

In [159]:
users = corpus.groupby(F.col('user_id')).agg(F.mean(F.col('stars')))

In [160]:
users.show()

Py4JJavaError: An error occurred while calling o1336.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 116.0 failed 1 times, most recent failure: Lost task 0.0 in stage 116.0 (TID 2296, localhost, executor driver): java.lang.NullPointerException
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.agg_doAggregateWithKeys$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:377)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:231)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:225)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:826)
	at 

In [ ]:
users = corpus.groupby('user_id').agg({ 'stars': 'mean', 'text' :'count'})

In [220]:
users = corpus.groupby('user_id').agg(F.mean('stars'), F.count('text'))

In [179]:
corpus.selecdescribe().show()

+-------+--------------------+--------------------+------------------+------------------+--------------------+
|summary|         business_id|                text|             stars|           popular|             user_id|
+-------+--------------------+--------------------+------------------+------------------+--------------------+
|  count|               10000|               10000|             10000|             10000|               10000|
|   mean|                null|                null|            3.8152|            1.7687|                null|
| stddev|                null|                null|1.3878910563003197|4.7908762830567975|                null|
|    min|-6O_cneh21YEljiDh...|!!!!!

We found o...|                 1|                 0|--3WaS23LcIXtxyFU...|
|    max|zlmJrXHH02opGCUya...|日曜日のランチで行きました。
エッ...|                 5|               196|zzanZzhoA3wufshSr...|
+-------+--------------------+--------------------+------------------+------------------+--------------------+



In [191]:
testjf = corpus.select(F.col('user_id'), F.col('stars'))

testjf.cache()
testjf.show()

+--------------------+-----+
|             user_id|stars|
+--------------------+-----+
|np2cEu_LDw4R8Xg7s...|    5|
|3YJr9-Tvx0oUSJjOV...|    5|
|rxMYqoGStBo0VdNaF...|    5|
|Tt1wHqPLZwK6271TR...|    5|
|mR1qquQNrkJUbDP05...|    1|
|_yiSOF_pYgTmH43TT...|    5|
|8zFz4AKZlnihFHkbg...|    5|
|Uz_drvjFs7haf3PtW...|    5|
|3qAudxedWMgIboNlK...|    5|
|9zkY0oadZgZ8zqp5i...|    5|
|cgAhU-Vi4yV2ncQOf...|    5|
|b3vRnnSNPhpI4w6u3...|    5|
|xRDpIFYtTpUZE0kY4...|    5|
|S8Tk9LCAsCSgBnKNn...|    5|
|ydNngW5aPPEInNpqp...|    5|
|PTHVJ3XQYENrz0V17...|    5|
|Q7xxvMVhns_CZ7G0b...|    5|
|WQk93K0kb_R3GiODn...|    5|
|btToRnA9eWO-xrqBa...|    5|
|J2o_DNR1JFTTz86xI...|    5|
+--------------------+-----+
only showing top 20 rows



In [224]:
users = testjf.groupBy(F.col('user_id')).agg(F.count('user_id').alias('count'),\
                                     F.mean('stars').alias('avg_rating')).sort('avg_rating',ascending=False)

In [225]:
ucount = users.filter('count > 5')
ucount.show()

+--------------------+-----+-----------------+
|             user_id|count|       avg_rating|
+--------------------+-----+-----------------+
|CxDOIDnH8gp9KXzpB...|    7|3.142857142857143|
+--------------------+-----+-----------------+



In [197]:
spark.sql('''
    SELECT user_id, count(user_id) as count, mean(stars)
    FROM corpus
    GROUP BY user_id
    LIMIT 10
''').show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



Py4JJavaError: An error occurred while calling o2165.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 160.0 failed 1 times, most recent failure: Lost task 0.0 in stage 160.0 (TID 2736, localhost, executor driver): java.lang.NullPointerException
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.agg_doAggregateWithKeys$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:377)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:231)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:225)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:826)
	at 

In [80]:
spark.udf.register('labels', lambda x: 0 if x < 4 else 1)

In [ ]:
spark.udf.register('binary', lambda x:)

In [95]:
spark.udf.register('length', len)

In [90]:
corpus_l = spark.sql('''
            SELECT text, labels(stars) as label
            FROM corpus
            WHERE popular > 0
''')